In [54]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import statsmodels.api as sm

from patsy import dmatrices
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from gensim import corpora, models, similarities


In [55]:
df = pd.read_csv('./saved_models/merged_df_bills_topics_50.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,42,43,44,45,46,47,48,49,bill_id,bill_status
0,0,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039086,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
1,1,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039086,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
2,2,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039090,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
3,3,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039090,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
4,4,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039093,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB13,0


In [57]:
#remove wierd index columns
df2 = df.iloc[:, 1:-2] 
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039086,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
1,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039086,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
2,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039090,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
3,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039090,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
4,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.039093,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001


In [58]:
# get the right rows for the models
x = df2.values
xScaled = StandardScaler().fit_transform(x)

y = df['bill_status'].values.astype('int')

In [59]:
print 'starting starting'
X_train, X_test, y_train, y_test = train_test_split(xScaled, y, 
    test_size=0.33, random_state=42)


starting starting


In [37]:
#df[df['bill_status'].isnull()].count()

In [60]:
print 'logit statsmodel'
model = sm.Logit(df['bill_status'],df.iloc[:, 2:-2] )
result = model.fit()


logit statsmodel
Optimization terminated successfully.
         Current function value: 0.578613
         Iterations 8


In [61]:
print result.summary()


                           Logit Regression Results                           
Dep. Variable:            bill_status   No. Observations:                13569
Model:                          Logit   Df Residuals:                    13520
Method:                           MLE   Df Model:                           48
Date:                Sun, 09 Oct 2016   Pseudo R-squ.:                 0.05341
Time:                        21:10:22   Log-Likelihood:                -7851.2
converged:                       True   LL-Null:                       -8294.1
                                        LLR p-value:                1.272e-154
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
1             -0.3242      2.037     -0.159      0.874        -4.316     3.668
2             -3.5595      1.401     -2.540      0.011        -6.306    -0.813
3             -2.0421      1.542     -1.324      0.1

In [51]:
#print result.conf_int()


In [62]:
print np.exp(result.params)

1       0.723103
2       0.028452
3       0.129759
4       0.006510
5       0.007667
6       0.578860
7       0.837655
8       0.742147
9       0.006021
10      1.400890
11      1.149210
12      0.625645
13      0.319898
14      0.258622
15      0.026684
16      0.000168
17      0.133220
18      0.021212
19      0.234244
20      0.000129
21      0.079018
22      0.293075
23      0.029442
24      0.010734
25      0.873364
26      0.019411
27      0.480116
28      0.143753
29      0.017858
30      0.035051
31      0.141230
32      0.026745
33      1.046822
34      4.745947
35      0.471256
36     10.216894
37      1.962699
38      0.626138
39      0.192155
40      3.003595
41      0.076743
42      0.037870
43    194.947401
44      1.648731
45      0.221681
46      0.040742
47      1.004472
48      0.617035
49      1.605780
dtype: float64


In [63]:
params = result.params
conf = result.conf_int()
conf['OddsRation'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
table = np.exp(conf)
table_lr_output = pd.DataFrame(table).reset_index()
print table_lr_output.head()

  index      2.5%      97.5%        OR
0     1  0.013352  39.161289  0.723103
1     2  0.001825   0.443672  0.028452
2     3  0.006313   2.667203  0.129759
3     4  0.000036   1.184326  0.006510
4     5  0.000498   0.118154  0.007667


In [64]:
lda_tfidf_model = models.LdaModel.load('./saved_models/lda_tfidf_model_50.pkl')

def get_topic_num(num):
    return lda_tfidf_model.print_topic(num)

table_lr_output['topics'] = table_lr_output['index'].apply( lambda x: get_topic_num(int(x)))

In [65]:
table_lr_output.sort('OR', inplace=True)
table_lr_output.to_csv('./saved_models/LR_Output_50.csv')

In [68]:
table_lr_output

,index,2.5%,97.5%,OR,topics
19,20,0.000046,0.000360,0.000129,0.106*budget + 0.036*proclamation + 0.026*emer...
15,16,0.000001,0.023901,0.000168,0.009*salmon + 0.006*innovation + 0.005*hatche...
8,9,0.000289,0.125370,0.006021,0.047*firearm + 0.023*handgun + 0.016*korean +...
3,4,0.000036,1.184326,0.006510,0.012*annuity + 0.010*dickinson + 0.007*presid...
4,5,0.000498,0.118154,0.007667,0.023*meal + 0.009*memorandum + 0.009*stigma +...
23,24,0.000171,0.674740,0.010734,0.011*vietnamese + 0.010*vietnam + 0.009*tissu...
28,29,0.004107,0.077641,0.017858,0.019*sale + 0.018*tax + 0.017*tangible + 0.01...
25,26,0.002570,0.146592,0.019411,0.038*water + 0.013*drinking + 0.012*groundwat...
17,18,0.000422,1.066660,0.021212,0.012*card + 0.009*cardholder + 0.006*firework...
14,15,0.000184,3.878668,0.026684,0.045*cancer + 0.019*prostate + 0.015*breast +...
